In [24]:
from collections import Counter
import random

import pyarrow.dataset as ds
import duckdb
import pandas as pd

INDICADORES = {
    "AFD": "Adequação da Formação Docente",
    "ICG": "Complexidade de Gestão da Escola",
    "IED": "Esforço Docente",
    "ATU": "Média de Alunos por Turma",
    "HAD": "Média de Horas-aula diária",
    "DSU": "Percentual de Docentes com Curso Superior",
    "TDI": "Taxas de Distorção Idade-série"
}

DIMENSIONS = {
    "Ano": "NU_ANO_CENSO",
    "Nome da região geográfica": "NO_REGIAO",
    "Nome da Unidade da Federação": "NO_UF",
    "Nome da Mesorregião": "NO_MESORREGIAO",
    "Nome da Microrregião": "NO_MICRORREGIAO"
}

con = duckdb.connect('../../file.db')

microdados = ds.dataset("../../data/transformed/microdados.parquet", format="parquet", partitioning="hive")
con.register("microdados", microdados)

indicadores = {}
for indicador in INDICADORES:
    indicadores[indicador] = ds.dataset(f"../../data/transformed/indicadores/{indicador}.parquet", format="parquet", partitioning="hive")
    con.register(indicador, indicadores[indicador])

In [2]:
con.execute(f"""
    select
        *
    from AFD
    limit 100
    """
).df()

,NO_REGIAO,SG_UF,CO_MUNICIPIO,NO_MUNICIPIO,CO_ENTIDADE,NO_CATEGORIA,NO_DEPENDENCIA,GRUPO,METRICA,NU_ANO_CENSO
0,Norte,RO,1100015,Alta Floresta D'Oeste,11022558,Rural,Estadual,AFD_INF1,NaN,2016
1,Norte,RO,1100015,Alta Floresta D'Oeste,11024275,Urbana,Estadual,AFD_INF1,NaN,2016
2,Norte,RO,1100015,Alta Floresta D'Oeste,11024291,Rural,Municipal,AFD_INF1,NaN,2016
3,Norte,RO,1100015,Alta Floresta D'Oeste,11024372,Urbana,Municipal,AFD_INF1,50.0,2016
4,Norte,RO,1100015,Alta Floresta D'Oeste,11024593,Rural,Municipal,AFD_INF1,NaN,2016
...,...,...,...,...,...,...,...,...,...,...
95,Norte,RO,1100049,Cacoal,11025999,Rural,Municipal,AFD_INF1,100.0,2016
96,Norte,RO,1100049,Cacoal,11026014,Urbana,Estadual,AFD_INF1,NaN,2016
97,Norte,RO,1100049,Cacoal,11026073,Urbana,Estadual,AFD_INF1,NaN,2016
98,Norte,RO,1100049,Cacoal,11026197,Urbana,Estadual,AFD_INF1,NaN,2016


In [21]:
# quantidade de matriculas
dimension = "NU_ANO_CENSO"
con.execute(f"""
    select
        {dimension},
        cast(sum(QT_MAT_INF) as bigint) as 'Educação Básica',

        cast(sum(QT_MAT_FUND) as bigint) as 'Educação Infantil',
        cast(sum(QT_MAT_MED) as bigint) as 'Ensino Fundamental',
        cast(sum(QT_MAT_PROF) as bigint) as 'Ensino Médio',
        cast(sum(QT_MAT_PROF_TEC) as bigint) as 'Educação Profissional',
        cast(sum(QT_MAT_EJA) as bigint) as 'Educação de Jovens e Adultos (EJA)',
        cast(sum(QT_MAT_ESP) as bigint) as 'Educação Especial'
    from microdados
    group by {dimension}
    """
).df()

,NU_ANO_CENSO,Educação Básica,Educação Infantil,Ensino Fundamental,Ensino Médio,Educação Profissional,Educação de Jovens e Adultos (EJA),Educação Especial
0,2016,8279104,27691478,8133040,1859940,1775324,3482174,971372
1,2017,8508731,27348080,7930384,1831003,1791806,3598716,1066446
2,2019,8972778,26923730,7465891,1914749,1874974,3273668,1250967
3,2021,8319399,26515601,7770557,1892458,1851541,2962322,1350921
4,2018,8745184,27183970,7709929,1903230,1868917,3545988,1181276
5,2020,8829795,26718830,7550753,1936094,1901477,3002749,1308900


In [29]:
# quantidade de escolas
dimension = "Ano"
df = con.execute(f"""
    select
        {DIMENSIONS[dimension]} as {dimension},
        count(CO_ENTIDADE) as 'Quantidade de escolas'
    from microdados
    group by {DIMENSIONS[dimension]}
    """
).df()

In [30]:
df

,Ano,Quantidade de escolas
0,2016,237506
1,2019,228521
2,2017,236481
3,2018,236460
4,2020,224229
5,2021,221140


In [ ]:
df.melt(
        id_vars=[dimension],
        var_name="Nível de ensino",
        value_name="Número de matriculas",
        value_vars=df.columns[1:]
     )

In [14]:
con.execute(f"""
    select
        *
        from TDI
        limit 100
    """
).df()

,NO_REGIAO,SG_UF,CO_MUNICIPIO,NO_MUNICIPIO,CO_ENTIDADE,NO_CATEGORIA,NO_DEPENDENCIA,GRUPO,METRICA,NU_ANO_CENSO
0,Norte,RO,1100015,Alta Floresta D'Oeste,11022558,Rural,Estadual,TDI_FUN,16.7,2016
1,Norte,RO,1100015,Alta Floresta D'Oeste,11024291,Rural,Municipal,TDI_FUN,10.5,2016
2,Norte,RO,1100015,Alta Floresta D'Oeste,11024372,Urbana,Municipal,TDI_FUN,24.5,2016
3,Norte,RO,1100015,Alta Floresta D'Oeste,11024593,Rural,Municipal,TDI_FUN,5.9,2016
4,Norte,RO,1100015,Alta Floresta D'Oeste,11024666,Rural,Municipal,TDI_FUN,17.3,2016
...,...,...,...,...,...,...,...,...,...,...
95,Norte,RO,1100049,Cacoal,11026839,Urbana,Municipal,TDI_FUN,12.9,2016
96,Norte,RO,1100049,Cacoal,11026936,Rural,Municipal,TDI_FUN,20.4,2016
97,Norte,RO,1100049,Cacoal,11026960,Rural,Municipal,TDI_FUN,14.7,2016
98,Norte,RO,1100049,Cacoal,11027002,Urbana,Municipal,TDI_FUN,5.5,2016


In [20]:
dimension = "SG_UF"
indicador_grupo = "TDI"
con.execute(f"""
    select
        m.{dimension},
        i.GRUPO,
        mean(i.METRICA)
        from microdados m
        left join {indicador_grupo} i
            on m.NU_ANO_CENSO = i.NU_ANO_CENSO
            and m.CO_ENTIDADE = i.CO_ENTIDADE
        where i.GRUPO like '{indicador_grupo}%'
        group by m.{dimension}, i.GRUPO
        order by 2, 3 desc
    """
).df()

,SG_UF,GRUPO,"mean(i.""METRICA"")"
0,AC,TDI_F00,13.243979
1,RR,TDI_F00,8.535126
2,AP,TDI_F00,8.206277
3,AM,TDI_F00,7.645357
4,PA,TDI_F00,7.639649
...,...,...,...
454,GO,TDI_MED,21.811849
455,PR,TDI_MED,20.738421
456,SC,TDI_MED,19.085700
457,DF,TDI_MED,19.069469
